This notebook provides code to train model on the airbus dataset

In [ ]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

# Enter the foldername in your Drive where you have saved the unzipped
# assignment folder, e.g. 'cs231n/assignments/assignment1/'
FOLDERNAME = 'test_archi_IA'
assert FOLDERNAME is not None, "[!] Enter the foldername."

# Now that we've mounted your Drive, this ensures that
# the Python interpreter of the Colab VM can load
# python files from within it.
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive/


Imports

In [ ]:
import numpy as np
import cv2
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
import tensorflow as tf
from time import time
import math
from tensorflow import keras

from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, ReLU, BatchNormalization, add,Softmax, AveragePooling2D, Dense, Input, GlobalAveragePooling2D,Add
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers.schedules import ExponentialDecay


In [ ]:
#extraction de la dataset
import pathlib
import os
import zipfile
from zipfile import ZipFile

# training_dataset_file_name = "./drive/MyDrive/transfer_learning_mobilenet/dataset_airbus_240.zip"
# with ZipFile(training_dataset_file_name, 'r') as zip:
#       zip.extractall('./drive/MyDrive/transfer_learning_mobilenet/')

data_dir_train = pathlib.Path('./drive/MyDrive/transfer_learning_mobilenet/dataset_airbus/dataset_airbus') # airbus
print(os.path.abspath(data_dir_train))
data_dir_test = pathlib.Path('./drive/MyDrive/test_archi_IA/part_of_dataset_airbus_test')
print(os.path.abspath(data_dir_test))

/content/drive/MyDrive/transfer_learning_mobilenet/dataset_airbus/dataset_airbus
/content/drive/MyDrive/test_archi_IA/part_of_dataset_airbus_test


Import training dataset (airbus)

In [ ]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
list_train = os.listdir('./drive/MyDrive/transfer_learning_mobilenet/dataset_airbus/dataset_airbus')#airbus


n_classes = 2
image_height = 240
image_width = 240
n_channel = 3
batch_size = 20

train_dataset = image_dataset_from_directory(
    directory=data_dir_train,
    subset = 'training',
    shuffle = True,
    validation_split = 0.2,
    seed=42,
    batch_size=batch_size,
    image_size=(image_height,image_width))

class_names_train = list_train
print(class_names_train)


validation_dataset = image_dataset_from_directory(
    directory=data_dir_train,
    subset = 'validation',
    shuffle = True,
    validation_split = 0.2,
    seed=42,
    batch_size=batch_size,
    image_size=(image_height,image_width))

class_names = validation_dataset.class_names
print(class_names)

Found 75069 files belonging to 2 classes.
Using 60056 files for training.
['bateau', 'pas_bateau']
Found 75069 files belonging to 2 classes.
Using 15013 files for validation.
['bateau', 'pas_bateau']


Import your test dataset

In [ ]:
"""test_file_name = "./drive/MyDrive/code_laetitia/test_dataset.zip"
with ZipFile(test_file_name, 'r') as zip:
    zip.extractall('./drive/MyDrive/code_laetitia/')"""

list_test = os.listdir('./drive/MyDrive/test_archi_IA/part_of_dataset_airbus_test')
test_dataset = image_dataset_from_directory(
    directory=data_dir_test,
    seed=42,
    batch_size=batch_size,
    image_size=(image_height,image_width))

class_names_test = list_test
print(class_names_test)

Found 121 files belonging to 2 classes.
['bateau', 'pas_bateau']


Dataset augmention for improved training

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.Rescaling(1./127.5, offset=-1)

Prepare your model

In [ ]:
conv2D = tf.keras.layers.Conv2D(64, (3,3), activation = 'relu')
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()


inputs = tf.keras.Input(shape=(240, 240, 3))
network = rescale(inputs)

network = tf.keras.layers.Conv2D(8, (3,3), padding='same', activation='relu')(network)
network = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(network)

network = tf.keras.layers.Conv2D(16, (3,3), padding='same', activation='relu')(network) 
network = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(network)

network = tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu')(network)
network = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(network)

network = tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu')(network)
network = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(network)

network = tf.keras.layers.Flatten()(network)
network = tf.keras.layers.Dense(32, activation='relu')(network)
outputs = tf.keras.layers.Dense(2, activation = 'softmax')(network)
model = tf.keras.Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 240, 240, 3)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 240, 240, 8)       224       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 120, 120, 8)      0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 120, 120, 16)      1168      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 60, 60, 16)       0         
 2D)                                                         

Weights table shape

In [ ]:
weights=model.get_weights()#return a numpy list of weights
for i in range(12):
  print(weights[i].shape)

(3, 3, 3, 8)
(8,)
(3, 3, 8, 16)
(16,)
(3, 3, 16, 32)
(32,)
(3, 3, 32, 32)
(32,)
(7200, 32)
(32,)
(32, 2)
(2,)


Path to the checkpoint repository

In [ ]:
checkpoint_path = "drive/MyDrive/test_archi_IA/checkpoints_airbus_new/weights.12-0.8841.cpkt"
checkpoint_dir = os.path.dirname(checkpoint_path)

If you have a valid checkpoint you can load it here

In [ ]:
model.load_weights(checkpoint_path)

Use algorithm optimization to increase training speed

In [ ]:
assert(tf.test.gpu_device_name())
tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(True) # Enable XLA.

Compile the model

In [ ]:
base_learning_rate = 0.00001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy (from_logits=False),
              metrics=['accuracy'])


First batch of training.  
Checkpoints are saved automatically when validation accuracy increase over the threshold.

In [ ]:
initial_epochs = 5
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="drive/MyDrive/test_archi_IA/checkpoints_airbus_new/weights.{epoch:02d}-{val_accuracy:.4f}.cpkt",
                                                 save_weights_only=True,
                                                 monitor='val_accuracy',
                                                 save_best_only=True,
                                                 mode='max',
                                                 initial_value_threshold = 0.8741,
                                                 verbose=1)
history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset,
                    callbacks=cp_callback
                    )

Epoch 1/5
3003/3003 [==============================] - ETA: 0s - loss: 0.3640 - accuracy: 0.8440
Epoch 1: val_accuracy improved from 0.84090 to 0.84314, saving model to drive/MyDrive/test_archi_IA/checkpoints_airbus_new/weights.01-0.8431.cpkt
3003/3003 [==============================] - 6128s 2s/step - loss: 0.3640 - accuracy: 0.8440 - val_loss: 0.3773 - val_accuracy: 0.8431
Epoch 2/5
3003/3003 [==============================] - ETA: 0s - loss: 0.3618 - accuracy: 0.8461
Epoch 2: val_accuracy did not improve from 0.84314
3003/3003 [==============================] - 164s 54ms/step - loss: 0.3618 - accuracy: 0.8461 - val_loss: 0.3762 - val_accuracy: 0.8427
Epoch 3/5
3000/3003 [============================>.] - ETA: 0s - loss: 0.3599 - accuracy: 0.8469
Epoch 3: val_accuracy improved from 0.84314 to 0.84487, saving model to drive/MyDrive/test_archi_IA/checkpoints_airbus_new/weights.03-0.8449.cpkt
3003/3003 [==============================] - 159s 53ms/step - loss: 0.3598 - accuracy: 0.8469 -

Next batch of training

In [ ]:
initial_epochs = 20

history = model.fit(train_dataset,
                    epochs=initial_epochs,
                    validation_data=validation_dataset,
                    callbacks=cp_callback
                    )

Epoch 1/20
3000/3003 [============================>.] - ETA: 0s - loss: 0.2759 - accuracy: 0.8861
Epoch 1: val_accuracy did not improve from 0.87757
3003/3003 [==============================] - 162s 54ms/step - loss: 0.2758 - accuracy: 0.8861 - val_loss: 0.3029 - val_accuracy: 0.8741
Epoch 2/20
3003/3003 [==============================] - ETA: 0s - loss: 0.2741 - accuracy: 0.8873
Epoch 2: val_accuracy improved from 0.87757 to 0.87917, saving model to drive/MyDrive/test_archi_IA/checkpoints_airbus_new/weights.02-0.8792.cpkt
3003/3003 [==============================] - 164s 54ms/step - loss: 0.2741 - accuracy: 0.8873 - val_loss: 0.2949 - val_accuracy: 0.8792
Epoch 3/20
3003/3003 [==============================] - ETA: 0s - loss: 0.2717 - accuracy: 0.8882
Epoch 3: val_accuracy did not improve from 0.87917
3003/3003 [==============================] - 161s 54ms/step - loss: 0.2717 - accuracy: 0.8882 - val_loss: 0.2982 - val_accuracy: 0.8782
Epoch 4/20
3000/3003 [============================

Evaluate the model on test dataset

In [ ]:
scores = model.evaluate(test_dataset, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

7/7 [==============================] - 1s 83ms/step - loss: 0.8433 - accuracy: 0.8347
accuracy: 83.47%


Here you can visualize the model predictions on a batch of images from the dataset

In [ ]:
# Retrieve a batch of images from the test set
image_batch, label_batch = test_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch)
print(predictions)
predictions = model.predict_on_batch(image_batch).flatten()
n = 0
results = {}
for i in range(0,10) :
  if  (max(predictions[n], predictions[n+1]) == predictions[n]) :
    results[i] = 0
  if (max(predictions[n], predictions[n+1]) == predictions[n+1]) :
    results[i] = 1
  n += 2 
print(results)

print('Predictions:\n', results)
print('Labels:\n', label_batch)

plt.figure(figsize=(10, 10))
for i in range(9):
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[results[i]])
  plt.axis("off")

Fine tuning on the model .To be done when accuracy reach celling on classic training (usually around 95%)

In [ ]:
model.trainable = True

Choose how many layer you want to freeze

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 9

# Freeze all the layers before the `fine_tune_at` layer
for layer in model.layers[:fine_tune_at]:
  layer.trainable = False

Number of layers in the base model:  13


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = 10

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath="drive/MyDrive/test_archi_IA/checkpoints_airbus/weights.{epoch:02d}-{val_accuracy:.4f}.cpkt",
                                                 save_weights_only=True,
                                                 monitor='val_accuracy',
                                                 save_best_only=True,
                                                 mode='max',
                                                 initial_value_threshold = 0.91074,
                                                 verbose=1)

history = model.fit(train_dataset,
                    epochs=fine_tune_epochs,
                    validation_data=validation_dataset,
                    callbacks = cp_callback
                    )

Epoch 1/10
3003/3003 [==============================] - ETA: 0s - loss: 0.1057 - accuracy: 0.9625

Evaluate model after fine tuning

In [ ]:
scores = model.evaluate(test_dataset, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))